# Lab 3: Azure AI Foundry Agent Service 기반 Multi-Agent 구축

## 개요 (Overview)

이 노트북에서는 Azure AI Foundry의 Agent Service를 활용하여 Multi-Agent 시스템을 구축하고 배포합니다.

### 아키텍처 (Architecture)

```
┌────────────────────────────────────────────────────────────┐
│                 Multi-Agent System                         │
│                                                            │
│  ┌─────────────────────────────────────────────┐          │
│  │          Main Agent                         │          │
│  │  (Task Analysis & Routing)                  │          │
│  └────────────┬────────────────┬────────────────┘          │
│               │                │                           │
│       ┌───────▼──────┐  ┌──────▼──────────┐               │
│       │  Tool Agent  │  │  Research       │               │
│       │  (MCP)       │  │  Agent (RAG)    │               │
│       └──────┬───────┘  └────────┬────────┘               │
│              │                   │                         │
│       ┌──────▼───────┐    ┌──────▼─────────┐              │
│       │  MCP Server  │    │  Azure AI      │              │
│       │  (ACA)       │    │  Search (RAG)  │              │
│       └──────────────┘    └────────────────┘              │
└────────────────────────────────────────────────────────────┘
```

### 주요 구성 요소 (Components)

1. **Main Agent**: 사용자 요청을 분석하고 적절한 Agent로 라우팅
2. **Tool Agent**: MCP 서버의 도구들을 활용 (날씨, 계산기 등)
3. **Research Agent**: Azure AI Search를 통한 지식 베이스 검색
4. **MCP Server**: Azure Container Apps에 배포된 도구 서버

### Python 모듈 구조 (Python Modules)

```
src/agent/
├── main_agent.py       - Main Agent 클래스
├── tool_agent.py       - Tool Agent 클래스 (MCP)
├── research_agent.py   - Research Agent 클래스 (RAG)
└── requirements.txt    - Python 의존성
```

### 학습 목표 (Learning Objectives)

1. ✅ Azure AI Foundry Agent Service 이해 및 활용
2. ✅ MCP Server를 Azure Container Apps에 배포
3. ✅ Connected Agent를 사용한 MCP 연동
4. ✅ Multi-Agent 오케스트레이션 패턴 구현
5. ✅ RAG 기반 Agent 구축
6. ✅ Agent 간 협업 및 응답 통합

## 1. 환경 설정 및 인증 (Setup & Authentication)

In [162]:
import sys, subprocess, os, json
import platform

# 운영체제에 따라 PATH 설정
system = platform.system()
if system == 'Darwin':  # macOS
    extra_paths = '/opt/homebrew/bin:/usr/local/bin'
elif system == 'Linux':  # Linux / Codespaces
    extra_paths = '/usr/local/bin:/usr/bin:/home/codespace/.local/bin'
else:  # Windows
    extra_paths = ''

if extra_paths:
    os.environ['PATH'] = extra_paths + ':' + os.environ.get('PATH', '')

def check(cmd, name):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, timeout=3, env=os.environ)
        print(f"{'✓' if result.returncode == 0 else '✗'} {name}")
    except Exception as e:
        print(f"✗ {name}")

print("=== Prerequisites Check ===")
print(f"✓ Python {sys.version.split()[0]} ({system})")
check("az --version", "Azure CLI")
check("docker --version", "Docker")
print("="*50)

=== Prerequisites Check ===
✓ Python 3.13.7 (Darwin)
✗ Azure CLI
✓ Docker


In [163]:
# Azure 인증
print("=== Azure Authentication ===\n🔐 Authenticating...\n")

az = subprocess.run(
    "az login --tenant 16b3c013-d300-468d-ac64-7eda0820b6d3", 
    shell=True, 
    capture_output=True, 
    text=True
)
print(f"{'✅' if az.returncode == 0 else '❌'} Azure CLI")
print("="*50)

=== Azure Authentication ===
🔐 Authenticating...

✅ Azure CLI


In [164]:
# 설정 파일 로드
config_path = "config.json"
with open(config_path) as f:
    config = json.load(f)

# 환경 변수 설정
RESOURCE_GROUP = config["resource_group"]
LOCATION = config["location"]
PROJECT_CONNECTION_STRING = config["project_connection_string"]
SEARCH_ENDPOINT = config["search_endpoint"]
SEARCH_INDEX = config["search_index"]
CONTAINER_REGISTRY = config["container_registry_endpoint"]
CONTAINER_ENV_ID = config["container_apps_environment_id"]

# PROJECT_CONNECTION_STRING을 간단한 형식으로 변환
# config.json 형식: https://xxx/api/projects/yyy;subscription_id=zzz;resource_group=www
# 필요한 형식: https://xxx/api/projects/yyy (세미콜론 이후 제거)
simple_project_conn = PROJECT_CONNECTION_STRING.split(';')[0] if PROJECT_CONNECTION_STRING else ""

print("=== Configuration Loaded ===")
print(f"Resource Group: {RESOURCE_GROUP}")
print(f"Location: {LOCATION}")
print(f"Search Index: {SEARCH_INDEX}")
print(f"Container Registry: {CONTAINER_REGISTRY}")
print("="*50)

=== Configuration Loaded ===
Resource Group: rg-aiagent-ciid4s
Location: eastus
Search Index: ai-agent-knowledge-base
Container Registry: crpf3kkfblz2ryy.azurecr.io


## 2. Azure AI Search 키 가져오기 (Get Search Key)

RAG Agent가 사용할 Azure AI Search 관리 키를 가져옵니다.

In [165]:
# AI Search 관리 키 가져오기
search_name = config["search_service_name"]

search_key_cmd = f"""
az search admin-key show \
    --resource-group {RESOURCE_GROUP} \
    --service-name {search_name} \
    --query primaryKey -o tsv
"""

result = subprocess.run(search_key_cmd, shell=True, capture_output=True, text=True)
SEARCH_KEY = result.stdout.strip()

if SEARCH_KEY:
    print(f"✅ Search key retrieved: {SEARCH_KEY[:10]}...")
    os.environ['SEARCH_KEY'] = SEARCH_KEY
else:
    print("❌ Failed to retrieve search key")

✅ Search key retrieved: 7xQ3Bu7vXI...


## 3. Azure AI Search 연결 추가 (Add Azure AI Search Connection)

Azure AI Foundry 프로젝트에 Azure AI Search 연결을 추가합니다.

**연결 추가 이유:**
- `AzureAISearchTool`은 프로젝트 연결을 통해 Azure AI Search에 액세스합니다
- 연결이 없으면 Research Agent가 RAG 기능 없이 일반 지식으로만 답변합니다

**작업 내용:**
- Azure AI Search 서비스 정보를 프로젝트에 연결로 등록
- 등록 후 Research Agent가 자동으로 연결을 사용하여 RAG 수행

In [166]:
# Azure AI Search 연결 확인
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.identity import DefaultAzureCredential

print("=== Checking Azure AI Search Connection ===\n")

# Initialize project client
print(f"🔍 Project: {simple_project_conn}\n")

project_client_for_connection = AIProjectClient(
    endpoint=simple_project_conn,
    credential=DefaultAzureCredential()
)

# Check existing connections
print("📋 Checking existing connections...\n")

connection_exists = False

try:
    # Try to get default Azure AI Search connection
    search_connection = project_client_for_connection.connections.get_default(
        connection_type=ConnectionType.AZURE_AI_SEARCH
    )
    print(f"✅ Default Azure AI Search connection found!")
    print(f"   Connection ID: {search_connection.id}")
    print(f"   Connection Name: {search_connection.name}\n")
    connection_exists = True
    
except Exception as e:
    print(f"⚠️  No default connection found: {e}\n")
    
    # Try to list all Azure AI Search connections
    try:
        print("🔍 Searching for any Azure AI Search connections...")
        connections = list(project_client_for_connection.connections.list(
            connection_type=ConnectionType.AZURE_AI_SEARCH
        ))
        
        if connections:
            print(f"✅ Found {len(connections)} Azure AI Search connection(s):\n")
            for conn in connections:
                print(f"   • {conn.name}")
                print(f"     ID: {conn.id}\n")
            connection_exists = True
        else:
            print("❌ No Azure AI Search connections found\n")
            
    except Exception as e2:
        print(f"❌ Error listing connections: {e2}\n")

# Display result
print("="*60)
if connection_exists:
    print("✅ Azure AI Search connection is configured!")
    print("\n💡 Research Agent will use RAG with this connection.")
    print("   You can now test the Research Agent.\n")
else:
    print("❌ No Azure AI Search connection found")
    print("\n📝 Please add the connection via Azure Portal:")
    print("   1. Go to: https://ai.azure.com")
    print(f"   2. Open project: {simple_project_conn.split('/')[-1]}")
    print("   3. Settings → Connections → + New connection")
    print("   4. Select: Azure AI Search")
    print(f"   5. Configure with endpoint: {SEARCH_ENDPOINT}\n")

print("="*60)

=== Checking Azure AI Search Connection ===

🔍 Project: https://aoai-pf3kkfblz2ryy.services.ai.azure.com/api/projects/proj-pf3kkfblz2ryy

📋 Checking existing connections...

✅ Default Azure AI Search connection found!
   Connection ID: /subscriptions/49a89096-a0ae-4e59-816b-dcb0a6fe9168/resourceGroups/rg-aiagent-ciid4s/providers/Microsoft.CognitiveServices/accounts/aoai-pf3kkfblz2ryy/projects/proj-pf3kkfblz2ryy/connections/srchpf3kkfblz2ryy
   Connection Name: srchpf3kkfblz2ryy

✅ Azure AI Search connection is configured!

💡 Research Agent will use RAG with this connection.
   You can now test the Research Agent.



## 4. MCP Server 배포 (Deploy MCP Server)

Model Context Protocol 서버를 Azure Container Apps에 배포합니다.

### MCP Server 기능
- `get_weather`: 도시별 날씨 정보 조회
- `calculate`: 수학 계산 수행
- `get_current_time`: 현재 시간 조회
- `generate_random_number`: 랜덤 숫자 생성

### 서버 구성
- **프로토콜**: HTTP REST API
- **포트**: 8000
- **엔드포인트**:
  - `/` - Root endpoint (health check)
  - `/health` - Health check
  - `/tools` - List available tools (GET)
  - `/tools/call` - Execute a tool (POST)

In [167]:
# Container Registry에 로그인 (재배포를 위해)
registry_name = CONTAINER_REGISTRY.split('.')[0]

print("=== Container Registry Login ===")
login_cmd = f"az acr login --name {registry_name}"
result = subprocess.run(login_cmd, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print(f"✅ Logged in to {registry_name}")
    print("\n💡 MCP 서버가 업데이트되었습니다. 다음 셀에서 재빌드하세요.")
else:
    print(f"❌ Login failed: {result.stderr}")
print("="*50)

=== Container Registry Login ===
✅ Logged in to crpf3kkfblz2ryy

💡 MCP 서버가 업데이트되었습니다. 다음 셀에서 재빌드하세요.


In [168]:
# .env 파일 생성 (MCP Server용 - 향후 확장성을 위해)
print("=== Creating .env file for MCP Server ===\n")

# MCP 서버는 현재 환경 변수가 필요하지 않지만, 향후 확장을 위해 빈 파일 생성
env_content = """# MCP Server Configuration
# Add any configuration variables here as needed

# Example: API keys, endpoints, etc.
# WEATHER_API_KEY=your_api_key_here
# EXTERNAL_SERVICE_URL=your_service_url_here
"""

env_file_path = "src/mcp/.env"

try:
    with open(env_file_path, 'w') as f:
        f.write(env_content)
    
    print(f"✅ Created {env_file_path}")
    print("\n💡 MCP 서버는 현재 환경 변수가 필요하지 않습니다.")
    print("   하지만 향후 외부 API 연동 시 이 파일에 설정을 추가할 수 있습니다.")
    
except Exception as e:
    print(f"❌ Failed to create .env file: {e}")

print("\n" + "="*60)

=== Creating .env file for MCP Server ===

✅ Created src/mcp/.env

💡 MCP 서버는 현재 환경 변수가 필요하지 않습니다.
   하지만 향후 외부 API 연동 시 이 파일에 설정을 추가할 수 있습니다.



In [169]:
# MCP Server 이미지 빌드 및 푸시
import time

mcp_image = f"{CONTAINER_REGISTRY}/mcp-server:latest"

print("=== Building MCP Server Image ===")
print(f"Image: {mcp_image}\n")

# 빌드 (Azure Container Apps용 linux/amd64 플랫폼)
build_cmd = f"docker build --platform linux/amd64 -t {mcp_image} ./src/mcp"
print("🔨 Building image (linux/amd64)...")
start_time = time.time()

result = subprocess.run(build_cmd, shell=True, capture_output=True, text=True)
elapsed = time.time() - start_time

if result.returncode == 0:
    print(f"✅ Build successful ({elapsed:.1f}s)")
else:
    print(f"❌ Build failed: {result.stderr}")
    
# 푸시
if result.returncode == 0:
    print("\n📤 Pushing image to registry...")
    push_cmd = f"docker push {mcp_image}"
    result = subprocess.run(push_cmd, shell=True, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✅ Push successful")
    else:
        print(f"❌ Push failed: {result.stderr}")

print("="*50)

=== Building MCP Server Image ===
Image: crpf3kkfblz2ryy.azurecr.io/mcp-server:latest

🔨 Building image (linux/amd64)...
✅ Build successful (2.1s)

📤 Pushing image to registry...
✅ Push successful


In [170]:
# MCP Server를 Container App으로 배포
mcp_app_name = "mcp-server"

print("=== Deploying MCP Server to ACA ===")
print(f"App Name: {mcp_app_name}\n")

deploy_cmd = f"""
az containerapp create \
    --name {mcp_app_name} \
    --resource-group {RESOURCE_GROUP} \
    --environment {CONTAINER_ENV_ID.split('/')[-1]} \
    --image {mcp_image} \
    --target-port 8000 \
    --ingress external \
    --min-replicas 1 \
    --max-replicas 3 \
    --cpu 0.5 \
    --memory 1.0Gi \
    --registry-server {CONTAINER_REGISTRY}
"""

print("🚀 Deploying...")
result = subprocess.run(deploy_cmd, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print("✅ Deployment successful")
    
    # Get endpoint
    show_cmd = f"""
    az containerapp show \
        --name {mcp_app_name} \
        --resource-group {RESOURCE_GROUP} \
        --query properties.configuration.ingress.fqdn -o tsv
    """
    result = subprocess.run(show_cmd, shell=True, capture_output=True, text=True)
    MCP_ENDPOINT = f"https://{result.stdout.strip()}"
    
    print(f"\n🌐 MCP Endpoint: {MCP_ENDPOINT}")
    
    # Update config
    config['mcp_endpoint'] = MCP_ENDPOINT
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
    print("✅ Config updated")
else:
    print(f"❌ Deployment failed: {result.stderr}")
    MCP_ENDPOINT = None

print("="*50)

=== Deploying MCP Server to ACA ===
App Name: mcp-server

🚀 Deploying...
✅ Deployment successful

🌐 MCP Endpoint: https://mcp-server.bluestone-09016d03.eastus.azurecontainerapps.io
✅ Config updated


## 5. Agent Container 빌드 및 배포 (Build & Deploy Agent Container)

Agent들을 컨테이너 이미지로 빌드하고 Azure Container Apps에 배포합니다.

### 컨테이너에 포함된 Agent 모듈
- `main_agent.py` - Main Agent 클래스
- `tool_agent.py` - Tool Agent 클래스 
- `research_agent.py` - Research Agent 클래스
- `server.py` - HTTP health check 서버

### 서버 구성
- **프레임워크**: FastAPI
- **포트**: 8000
- **엔드포인트**:
  - `/health` - Health check
  - `/info` - Service information
  - `/` - Root (returns health status)

### 배포 아키텍처
```
┌─────────────────────────────────────────┐
│   Azure Container Apps Environment      │
│                                         │
│  ┌──────────────┐  ┌────────────────┐  │
│  │  MCP Server  │  │  Agent Service │  │
│  │  Container   │  │  Container     │  │
│  │  :8000       │  │  :8000         │  │
│  └──────────────┘  └────────────────┘  │
└─────────────────────────────────────────┘
```

In [171]:
# .env 파일 생성 (Agent Container에 포함될 환경 변수)
print("=== Creating .env file for Agent Container ===\n")

# 1. Application Insights Connection String 가져오기
print("📊 Getting Application Insights connection string...")
appinsights_cmd = f"""
az monitor app-insights component show \
    --resource-group {RESOURCE_GROUP} \
    --query "[0].connectionString" -o tsv
"""

result = subprocess.run(appinsights_cmd, shell=True, capture_output=True, text=True)

if result.returncode == 0 and result.stdout.strip():
    APP_INSIGHTS_CONN_STR = result.stdout.strip()
    print(f"✅ Application Insights connection string retrieved\n")
else:
    print(f"⚠️  Could not get Application Insights connection string")
    print(f"   Error: {result.stderr}")
    print(f"   Proceeding without Application Insights (Analytics will not work)\n")
    APP_INSIGHTS_CONN_STR = ""

# 2. .env 파일 생성
env_content = f"""# Azure AI Foundry Configuration
PROJECT_CONNECTION_STRING={simple_project_conn}

# Azure AI Search Configuration
SEARCH_ENDPOINT={SEARCH_ENDPOINT}
SEARCH_KEY={SEARCH_KEY}
SEARCH_INDEX={SEARCH_INDEX}

# MCP Server Configuration
MCP_ENDPOINT={MCP_ENDPOINT if MCP_ENDPOINT else ''}

# Application Insights Configuration (for Application Analytics)
APPLICATIONINSIGHTS_CONNECTION_STRING={APP_INSIGHTS_CONN_STR}
"""

env_file_path = "src/agent/.env"

try:
    with open(env_file_path, 'w') as f:
        f.write(env_content)
    
    print(f"✅ Created {env_file_path}")
    print("\n📋 Environment variables:")
    for line in env_content.strip().split('\n'):
        if line and not line.startswith('#'):
            key = line.split('=')[0]
            print(f"   • {key}")
    
    print("\n💡 이 파일은 Docker 이미지에 포함됩니다.")
    print("   배포 시 별도의 환경 변수 설정이 필요하지 않습니다.")
    
    if APP_INSIGHTS_CONN_STR:
        print("\n✅ Application Insights 설정 완료!")
        print("   → Application Analytics에서 메트릭을 확인할 수 있습니다.")
    else:
        print("\n⚠️  Application Insights 미설정")
        print("   → Application Analytics는 작동하지 않지만 Agent는 정상 작동합니다.")
    
except Exception as e:
    print(f"❌ Failed to create .env file: {e}")

print("\n" + "="*60)

=== Creating .env file for Agent Container ===

📊 Getting Application Insights connection string...
✅ Application Insights connection string retrieved

✅ Created src/agent/.env

📋 Environment variables:
   • PROJECT_CONNECTION_STRING
   • SEARCH_ENDPOINT
   • SEARCH_KEY
   • SEARCH_INDEX
   • MCP_ENDPOINT
   • APPLICATIONINSIGHTS_CONNECTION_STRING

💡 이 파일은 Docker 이미지에 포함됩니다.
   배포 시 별도의 환경 변수 설정이 필요하지 않습니다.

✅ Application Insights 설정 완료!
   → Application Analytics에서 메트릭을 확인할 수 있습니다.



In [172]:
# Agent Container 이미지 빌드 및 푸시
import time

agent_image = f"{CONTAINER_REGISTRY}/agent-service:latest"

print("=== Building Agent Service Image ===")
print(f"Image: {agent_image}\n")

# 빌드 (Azure Container Apps용 linux/amd64 플랫폼)
build_cmd = f"docker build --platform linux/amd64 -t {agent_image} ./src/agent"
print("🔨 Building image (linux/amd64)...")
start_time = time.time()

result = subprocess.run(build_cmd, shell=True, capture_output=True, text=True)
elapsed = time.time() - start_time

if result.returncode == 0:
    print(f"✅ Build successful ({elapsed:.1f}s)")
    print(f"   Image contains: main_agent.py, tool_agent.py, research_agent.py")
else:
    print(f"❌ Build failed: {result.stderr}")
    
# 푸시
if result.returncode == 0:
    print("\n📤 Pushing image to registry...")
    push_cmd = f"docker push {agent_image}"
    result = subprocess.run(push_cmd, shell=True, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✅ Push successful")
    else:
        print(f"❌ Push failed: {result.stderr}")

print("="*50)

=== Building Agent Service Image ===
Image: crpf3kkfblz2ryy.azurecr.io/agent-service:latest

🔨 Building image (linux/amd64)...
✅ Build successful (1.4s)
   Image contains: main_agent.py, tool_agent.py, research_agent.py

📤 Pushing image to registry...
✅ Push successful


## 5.1. Azure 리소스 확인 (Verify Azure Resources)

Agent Service 배포 전에 필요한 Azure 리소스를 확인합니다.

**확인 항목:**
- ✅ Azure AI Project 리소스 ID
- ✅ AI Services (Cognitive Services) 리소스 ID

이 정보는 다음 배포 단계에서 Managed Identity에 권한을 자동으로 부여할 때 사용됩니다.

In [173]:
# Azure AI Project 및 AI Services 리소스 ID 확인
print("=== Verifying Azure Resources ===\n")

# 1. config.json에서 정보 가져오기 (이미 로드됨)
# config.json의 PROJECT_CONNECTION_STRING은 이미 간단한 형식:
# https://xxx.services.ai.azure.com/api/projects/yyy

# URL에서 project_name 추출
if '/api/projects/' in simple_project_conn:
    project_name = simple_project_conn.split('/api/projects/')[-1].strip()
else:
    project_name = None

print(f"📋 Project Information:")
print(f"   Resource Group: {RESOURCE_GROUP}")
print(f"   Project Name: {project_name if project_name else 'Not found in connection string'}\n")

# 2. AI Project 리소스 ID 가져오기
# Azure AI Foundry Project는 Microsoft.CognitiveServices/accounts/projects 타입
print("🔍 Finding AI Project resource...")
if project_name:
    # project_name을 포함하는 리소스 검색
    ai_project_cmd = f"""
    az resource list \
        --resource-group {RESOURCE_GROUP} \
        --query "[?contains(name, '{project_name}') && type=='Microsoft.CognitiveServices/accounts/projects'].id" -o tsv
    """
else:
    # 타입으로만 검색 (첫 번째 결과)
    ai_project_cmd = f"""
    az resource list \
        --resource-group {RESOURCE_GROUP} \
        --query "[?type=='Microsoft.CognitiveServices/accounts/projects'].id | [0]" -o tsv
    """

result = subprocess.run(ai_project_cmd, shell=True, capture_output=True, text=True)
if result.returncode == 0 and result.stdout.strip():
    ai_project_resource_id = result.stdout.strip()
    print(f"   ✅ AI Project Resource ID:")
    print(f"   {ai_project_resource_id}\n")
else:
    print(f"   ❌ Could not find AI Project")
    print(f"   Error: {result.stderr}\n")
    raise Exception("AI Project not found")

# 3. AI Services 리소스 ID 가져오기 (Cognitive Services account)
print("🔍 Finding AI Services (Cognitive Services) resource...")
ai_services_cmd = f"""
az resource list \
    --resource-group {RESOURCE_GROUP} \
    --resource-type Microsoft.CognitiveServices/accounts \
    --query "[0].id" -o tsv
"""

result = subprocess.run(ai_services_cmd, shell=True, capture_output=True, text=True)
if result.returncode == 0 and result.stdout.strip():
    ai_services_resource_id = result.stdout.strip()
    print(f"   ✅ AI Services Resource ID:")
    print(f"   {ai_services_resource_id}\n")
else:
    print(f"   ❌ Could not find AI Services")
    print(f"   Error: {result.stderr}\n")
    raise Exception("AI Services not found")

print("✅ All required resources verified!")
print("\n💡 다음 단계에서 이 리소스들에 권한을 부여합니다.")
print("="*60)

=== Verifying Azure Resources ===

📋 Project Information:
   Resource Group: rg-aiagent-ciid4s
   Project Name: proj-pf3kkfblz2ryy

🔍 Finding AI Project resource...
   ✅ AI Project Resource ID:
   /subscriptions/49a89096-a0ae-4e59-816b-dcb0a6fe9168/resourceGroups/rg-aiagent-ciid4s/providers/Microsoft.CognitiveServices/accounts/aoai-pf3kkfblz2ryy/projects/proj-pf3kkfblz2ryy

🔍 Finding AI Services (Cognitive Services) resource...
   ✅ AI Services Resource ID:
   /subscriptions/49a89096-a0ae-4e59-816b-dcb0a6fe9168/resourceGroups/rg-aiagent-ciid4s/providers/Microsoft.CognitiveServices/accounts/aoai-pf3kkfblz2ryy

✅ All required resources verified!

💡 다음 단계에서 이 리소스들에 권한을 부여합니다.


## 5.2. Agent Service 배포 및 권한 설정 (Deploy Agent Service with Permissions)

Agent Service를 배포하고 **배포 직후 자동으로** Managed Identity를 구성합니다.

**자동 수행 작업:**
1. ✅ Container App 배포
2. ✅ System-assigned Managed Identity 활성화
3. ✅ Azure AI User 역할 할당 (AI Project scope) ← agents/write 권한
4. ✅ 권한 전파 대기 및 Container 재시작

> 💡 **중요**: 배포와 권한 설정을 한 번에 처리하므로 완료까지 약 3-4분 소요됩니다.
> 
> ⚠️ **참고**: Managed Identity는 Container App이 생성된 후에만 활성화할 수 있으므로, 배포 직후 즉시 권한을 설정합니다.

In [174]:
# Agent Service를 Container App으로 배포 + Managed Identity 권한 설정
agent_app_name = "agent-service"

print("=== Deploying Agent Service to ACA ===")
print(f"App Name: {agent_app_name}\n")

print("💡 환경 변수는 이미 Docker 이미지에 포함되어 있습니다.")
print("   별도의 환경 변수 설정이 필요하지 않습니다.\n")

# 1. Container App 배포 (Managed Identity 포함, 권한 부여 전까지 replicas 0)
deploy_cmd = f"""
az containerapp create \
    --name {agent_app_name} \
    --resource-group {RESOURCE_GROUP} \
    --environment {CONTAINER_ENV_ID.split('/')[-1]} \
    --image {agent_image} \
    --target-port 8000 \
    --ingress external \
    --min-replicas 0 \
    --max-replicas 3 \
    --cpu 1.0 \
    --memory 2.0Gi \
    --registry-server {CONTAINER_REGISTRY} \
    --system-assigned \
"""

print("🚀 Deploying Agent Service with Managed Identity...")
print("   (Starting with 0 replicas to configure permissions first)")
result = subprocess.run(deploy_cmd, shell=True, capture_output=True, text=True, timeout=180)

if result.returncode == 0:
    print("✅ Deployment successful\n")
    
    # Get endpoint
    show_cmd = f"""
    az containerapp show \
        --name {agent_app_name} \
        --resource-group {RESOURCE_GROUP} \
        --query properties.configuration.ingress.fqdn -o tsv
    """
    result = subprocess.run(show_cmd, shell=True, capture_output=True, text=True)
    AGENT_ENDPOINT = f"https://{result.stdout.strip()}"
    
    print(f"🌐 Agent Endpoint: {AGENT_ENDPOINT}")
    
    # Update config
    config['agent_endpoint'] = AGENT_ENDPOINT
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
    print("✅ Config updated\n")
    

    # 2. Managed Identity Principal ID 가져오기
    print("="*60)
    print("🔐 Configuring Permissions\n")
    
    print("1️⃣ Getting Managed Identity Principal ID...")
    identity_cmd = f"""
    az containerapp show \
        --name {agent_app_name} \
        --resource-group {RESOURCE_GROUP} \
        --query identity.principalId -o tsv
    """
    
    result = subprocess.run(identity_cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0 and result.stdout.strip():
        principal_id = result.stdout.strip()
        print(f"   ✅ Principal ID: {principal_id}\n")
    else:
        print(f"   ❌ Failed to get Principal ID: {result.stderr}\n")
        raise Exception("Failed to get Managed Identity Principal ID")

    
    # 3. Azure AI User 역할 할당 (AI Project scope - agents/write 권한용)
    print("4️⃣ Assigning 'Azure AI User' role to AI Project...")
    print(f"   Scope: {ai_project_resource_id}")
    role_assignment_cmd = f"""
    az role assignment create \
        --assignee {principal_id} \
        --role "Azure AI User" \
        --scope {ai_project_resource_id}
    """
    
    result = subprocess.run(role_assignment_cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print("   ✅ Azure AI User role assigned (AI Project scope)\n")
    elif "already exists" in result.stderr.lower():
        print("   ✅ Azure AI User role already exists (AI Project scope)\n")
    else:
        print(f"   ❌ Role assignment FAILED!")
        print(f"   Error: {result.stderr}")
        print(f"   Return code: {result.returncode}\n")
    
    # 4. 권한 할당 검증
    print("5️⃣ Verifying role assignments...\n")
    import time
    time.sleep(5)  # 잠깐 대기 (역할 할당 API 완료 확인)
    
    role_check_cmd = f"""
    az role assignment list \
        --assignee {principal_id} \
        --query "[].{{role:roleDefinitionName, scope:scope}}" -o json
    """
    
    result = subprocess.run(role_check_cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        import json as json_lib
        current_roles = json_lib.loads(result.stdout)
        
        print(f"   📋 Current Role Assignments ({len(current_roles)} total):\n")
        
        # 필요한 역할 체크
        required_roles = {
            "Azure AI User (AI Project)": False
        }
        
        for role in current_roles:
            scope_parts = role['scope'].split('/')
            resource_name = scope_parts[-1] if scope_parts else 'Unknown'
            role_name = role['role']
            
            print(f"      • {role_name} → {resource_name}")
            
            # 필요한 역할인지 체크
            if role_name == "Azure AI User":
                if "projects" in role['scope'] or ai_project_resource_id in role['scope']:
                    required_roles["Azure AI User (AI Project)"] = True
        
        print(f"\n   🔍 Required Roles Verification:")
        all_roles_ok = True
        for role_name, assigned in required_roles.items():
            status = "✅" if assigned else "❌"
            print(f"      {status} {role_name}")
            if not assigned:
                all_roles_ok = False
        
        if all_roles_ok:
            print(f"\n   ✅ All required roles are assigned!\n")
        else:
            print(f"\n   ❌ Some required roles are missing!")
            print(f"      이 문제가 발생하면 Azure Portal에서 수동으로 권한을 확인하세요.\n")
    else:
        print(f"   ⚠️  Could not verify roles: {result.stderr}\n")
    
    # 5. 권한 전파 대기 안내
    print("="*60)
    print("6️⃣ Permissions assigned - waiting for propagation...\n")
    print("⚠️  Azure RBAC 권한 전파는 최대 5-10분 소요될 수 있습니다.")
    print("   Container는 replicas=0 상태로 유지됩니다.\n")
    
    print("📋 다음 단계:")
    print("   1. 위의 'Required Roles Verification'이 모두 ✅인지 확인")
    print("   2. 2-3분 정도 기다리세요")
    print("   3. 아래 셀(섹션 5.2.1)을 실행하여 Container를 시작하세요")
    print("   4. 만약 여전히 권한 오류가 발생하면:")
    print("      → 추가로 2-3분 더 기다린 후 섹션 5.2.1을 다시 실행하세요\n")
    
    print(f"💡 Principal ID (권한 확인용): {principal_id}\n")
    
    print("="*60)
    print("✅ Permissions configured successfully!")
    print(f"\n🌐 Endpoint: {AGENT_ENDPOINT}")
    print(f"\n📋 Assigned Roles:")
    print(f"   • Azure AI User (AI Project scope) ← agents/write 권한")
    print(f"\n⏳ 권한 전파를 기다린 후 다음 셀(4.4.1)을 실행하세요!")
        
else:
    print(f"❌ Deployment failed: {result.stderr}")

    AGENT_ENDPOINT = None
    AGENT_ENDPOINT = Noneprint("\n" + "="*60)


print("\n" + "="*60)

=== Deploying Agent Service to ACA ===
App Name: agent-service

💡 환경 변수는 이미 Docker 이미지에 포함되어 있습니다.
   별도의 환경 변수 설정이 필요하지 않습니다.

🚀 Deploying Agent Service with Managed Identity...
   (Starting with 0 replicas to configure permissions first)
✅ Deployment successful

🌐 Agent Endpoint: https://agent-service.bluestone-09016d03.eastus.azurecontainerapps.io
✅ Config updated

🔐 Configuring Permissions

1️⃣ Getting Managed Identity Principal ID...
   ✅ Principal ID: 3cfec86e-109f-4b56-8ecd-e5c95f2d5e55

4️⃣ Assigning 'Azure AI User' role to AI Project...
   Scope: /subscriptions/49a89096-a0ae-4e59-816b-dcb0a6fe9168/resourceGroups/rg-aiagent-ciid4s/providers/Microsoft.CognitiveServices/accounts/aoai-pf3kkfblz2ryy/projects/proj-pf3kkfblz2ryy
   ✅ Azure AI User role assigned (AI Project scope)

5️⃣ Verifying role assignments...

   📋 Current Role Assignments (0 total):


   🔍 Required Roles Verification:
      ❌ Azure AI User (AI Project)

   ❌ Some required roles are missing!
      이 문제가 발생하면 Azu

## 5.2.1. Agent Service 시작 (Start Agent Service)

권한 전파가 완료된 후 이 셀을 실행하여 Container를 시작합니다.

**실행 시점:**
- ⏰ 섹션 4.4 완료 후 **2-3분 대기**
- ⚠️ 권한 오류 발생 시: 추가로 2-3분 더 기다린 후 재실행

**수행 작업:**
- ✅ Container App을 replicas=1로 확장
- ✅ Container 시작 및 상태 확인

In [175]:
# replicas를 1로 확장
scale_cmd = f"""
az containerapp update \
    --name agent-service \
    --resource-group {RESOURCE_GROUP} \
    --min-replicas 1 \
    --max-replicas 1
"""

print("🚀 Scaling to 1 replica...")
result = subprocess.run(scale_cmd, shell=True, capture_output=True, text=True, timeout=120)

if result.returncode == 0:
    print("✅ Agent Service started successfully!")
    print(f"\n🌐 Endpoint: {AGENT_ENDPOINT}")
    print("\n💡 Container가 시작되는 데 약 30초 정도 소요됩니다.")
    print(f"   로그 확인: az containerapp logs show --name agent-service --resource-group {RESOURCE_GROUP} --tail 50")
else:
    print(f"❌ Failed to start: {result.stderr}")

print("\n" + "="*60)

🚀 Scaling to 1 replica...
✅ Agent Service started successfully!

🌐 Endpoint: https://agent-service.bluestone-09016d03.eastus.azurecontainerapps.io

💡 Container가 시작되는 데 약 30초 정도 소요됩니다.
   로그 확인: az containerapp logs show --name agent-service --resource-group rg-aiagent-ciid4s --tail 50



## 6. 🚀 배포된 Agent 테스트 (Test Deployed Agent via HTTP)

**중요한 차이점:**

이전 섹션 6, 7에서는 **노트북에서 로컬로 Agent를 생성**하여 테스트했습니다.
- ❌ 로컬 테스트 → Application Analytics에 **데이터가 나타나지 않습니다**

이 섹션에서는 **배포된 Container의 HTTP API**를 호출합니다.
- ✅ Container 테스트 → Application Analytics에 **데이터가 나타납니다**

**왜 차이가 날까요?**
- Application Analytics는 **Azure AI Foundry Project에서 실행된 Agent**만 추적합니다
- 노트북은 로컬 환경이므로 별도로 추적됩니다
- Container 내부의 Agent는 Project에 연결되어 있어 자동으로 추적됩니다

**테스트 방법:**
1. 배포된 Agent Service의 `/chat` 엔드포인트 호출
2. 다양한 질문으로 Agent 테스트
3. 5-10분 후 Application Analytics 확인

In [176]:
import requests
import json

print("=== 배포된 Agent Service 테스트 ===\n")

# Agent 엔드포인트 확인
if not AGENT_ENDPOINT:
    print("❌ AGENT_ENDPOINT가 설정되지 않았습니다!")
    print("   섹션 5.2를 먼저 실행하세요.\n")
else:
    print(f"🌐 Agent Endpoint: {AGENT_ENDPOINT}\n")
    
    # 1. Health check
    print("1️⃣ Health Check:")
    try:
        response = requests.get(f"{AGENT_ENDPOINT}/health", timeout=10)
        if response.status_code == 200:
            print(f"   ✅ Health: {response.json()}\n")
        else:
            print(f"   ❌ Health check failed: {response.status_code}\n")
    except Exception as e:
        print(f"   ❌ Error: {e}\n")
    
    # 2. Root endpoint 확인 (Agent 상태)
    print("2️⃣ Agent Status:")
    try:
        response = requests.get(f"{AGENT_ENDPOINT}/", timeout=10)
        if response.status_code == 200:
            status = response.json()
            print(f"   ✅ Status: {status.get('status')}")
            print(f"   📋 Agents:")
            for agent_name, available in status.get('agents', {}).items():
                icon = "✅" if available else "❌"
                print(f"      {icon} {agent_name}: {available}")
            print()
        else:
            print(f"   ❌ Status check failed: {response.status_code}\n")
    except Exception as e:
        print(f"   ❌ Error: {e}\n")
    
    print("="*70)
    print("\n💡 Agent Service가 정상 작동 중입니다!")
    print("   다음 셀에서 실제 질문을 테스트하세요.\n")
    print("="*70)

=== 배포된 Agent Service 테스트 ===

🌐 Agent Endpoint: https://agent-service.bluestone-09016d03.eastus.azurecontainerapps.io

1️⃣ Health Check:
   ✅ Health: {'status': 'healthy', 'service': 'Agent API Server'}

2️⃣ Agent Status:
   ✅ Status: running
   📋 Agents:
      ✅ main_agent: True
      ✅ tool_agent: True
      ✅ research_agent: True


💡 Agent Service가 정상 작동 중입니다!
   다음 셀에서 실제 질문을 테스트하세요.



### 6.1. Main Agent 테스트 (다양한 질문)

배포된 Main Agent에 다양한 질문을 보내서 Application Analytics 데이터를 생성합니다.

In [177]:
# 배포된 Main Agent에 다양한 질문 보내기
import requests
import json
import time

print("=== 배포된 Main Agent 테스트 ===\n")

# 테스트 케이스
test_cases = [
    {
        "message": "2 + 2는 얼마인가요?",
        "description": "간단한 계산 (Tool Agent 사용)"
    },
    {
        "message": "서울의 날씨를 알려주세요",
        "description": "날씨 조회 (Tool Agent 사용)"
    },
    {
        "message": "RAG 패턴에 대해 설명해주세요",
        "description": "기술 문서 검색 (Research Agent 사용)"
    },
    {
        "message": "Multi-agent orchestration의 best practice는 무엇인가요?",
        "description": "베스트 프랙티스 검색 (Research Agent 사용)"
    },
    {
        "message": "지금 몇 시인가요?",
        "description": "시간 조회 (Tool Agent 사용)"
    }
]

# 각 테스트 케이스 실행
success_count = 0
fail_count = 0

for i, test in enumerate(test_cases, 1):
    print(f"{'='*70}")
    print(f"[Test {i}/{len(test_cases)}]")
    print(f"질문: {test['message']}")
    print(f"설명: {test['description']}")
    print(f"{'='*70}")
    
    try:
        # POST 요청
        response = requests.post(
            f"{AGENT_ENDPOINT}/chat",
            json={"message": test['message']},
            headers={"Content-Type": "application/json"},
            timeout=60
        )
        
        if response.status_code == 200:
            result = response.json()
            print(f"\n✅ 응답 성공 (HTTP {response.status_code})")
            print(f"\n📝 Agent 응답:")
            print(f"{result.get('response', 'No response')[:300]}...")
            print()
            success_count += 1
        else:
            print(f"\n❌ 요청 실패 (HTTP {response.status_code})")
            print(f"오류: {response.text[:200]}")
            print()
            fail_count += 1
        
    except Exception as e:
        print(f"\n❌ 오류 발생: {e}")
        print()
        fail_count += 1
    
    # 다음 요청 전 잠시 대기 (rate limiting 방지)
    if i < len(test_cases):
        time.sleep(2)

print("="*70)
print(f"\n📊 테스트 결과:")
print(f"   ✅ 성공: {success_count}/{len(test_cases)}")
print(f"   ❌ 실패: {fail_count}/{len(test_cases)}")

if success_count > 0:
    print(f"\n🎉 {success_count}개의 Agent 호출이 성공했습니다!")
    print(f"\n💡 다음 단계:")
    print(f"   1. 5-10분 정도 기다리세요 (데이터 처리 시간)")
    print(f"   2. Azure AI Foundry 포털 접속: https://ai.azure.com")
    print(f"   3. 프로젝트 → '평가' → 'Application Analytics'")
    print(f"   4. 시간 범위를 '지난 24시간'으로 설정")
    print(f"   5. 메트릭 확인:")
    print(f"      • Total inference calls: {success_count}+ 건")
    print(f"      • Average inference call duration: X ms")
    print(f"      • Error rate: {fail_count}/{len(test_cases)}")
    print(f"\n   📋 참고: 더 많은 요청을 보낼수록 데이터가 더 빨리 나타납니다.")
    print(f"           이 셀을 여러 번 실행해도 됩니다!")
else:
    print(f"\n❌ 모든 요청이 실패했습니다.")
    print(f"   Container 로그를 확인하세요:")
    print(f"   az containerapp logs show --name agent-service --resource-group {RESOURCE_GROUP} --tail 50")

print("\n" + "="*70)

=== 배포된 Main Agent 테스트 ===

[Test 1/5]
질문: 2 + 2는 얼마인가요?
설명: 간단한 계산 (Tool Agent 사용)

✅ 응답 성공 (HTTP 200)

📝 Agent 응답:
2 + 2의 결과는 4입니다....

[Test 2/5]
질문: 서울의 날씨를 알려주세요
설명: 날씨 조회 (Tool Agent 사용)

✅ 응답 성공 (HTTP 200)

📝 Agent 응답:
현재 서울의 날씨 정보를 요청하고 있습니다. 곧 최신 정보를 제공해 드리겠습니다....

[Test 3/5]
질문: RAG 패턴에 대해 설명해주세요
설명: 기술 문서 검색 (Research Agent 사용)

✅ 응답 성공 (HTTP 200)

📝 Agent 응답:
RAG, 또는 Retrieval-Augmented Generation 패턴은 자연어 처리(NLP) 및 AI 모델 응용 프로그램에서 활용되는 강력한 방법론입니다. 이 패턴은 정보 검색과 생성 모델을 결합하여 더욱 정확하고 관련성 높은 응답을 생성하도록 돕습니다.

RAG 패턴의 핵심은 외부 데이터베이스나 문서에서 정보를 검색하는 프로세스를 포함함으로써, AI 모델에게 추가적인 컨텍스트와 데이터를 제공하는 것입니다. 이 패턴은 주로 다음의 두 가지 단계를 포함합니다:

1. **정보 검색 (Retrieval)**: 특정 쿼리에 대해, ...

[Test 4/5]
질문: Multi-agent orchestration의 best practice는 무엇인가요?
설명: 베스트 프랙티스 검색 (Research Agent 사용)

✅ 응답 성공 (HTTP 200)

📝 Agent 응답:
Multi-agent orchestration involves coordinating multiple agents to work together effectively. Here are some best practices to ensure successful implementation:

1. **Clear Agent Boundarie

### 6.2. 부하 테스트 (더 많은 데이터 생성)

Application Analytics에 더 많은 데이터를 빠르게 생성하려면 이 셀을 실행하세요.
여러 요청을 연속으로 보내서 메트릭이 더 빨리 나타나도록 합니다.

In [178]:
# 더 많은 요청을 보내서 Application Analytics 데이터 생성
import requests
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

print("=== Application Analytics 데이터 생성 (부하 테스트) ===\n")

# 간단한 질문들 (빠른 응답용)
quick_questions = [
    "1 + 1은?",
    "2 * 3은?",
    "10 - 5는?",
    "100 / 4는?",
    "현재 시간은?",
    "5 + 5는?",
    "3 * 7은?",
    "20 / 2는?",
]

def send_request(question, index):
    """단일 요청 전송"""
    try:
        response = requests.post(
            f"{AGENT_ENDPOINT}/chat",
            json={"message": question},
            headers={"Content-Type": "application/json"},
            timeout=30
        )
        return (index, response.status_code, question, True)
    except Exception as e:
        return (index, 0, question, False)

# 병렬 요청 전송
print(f"📤 {len(quick_questions)}개의 요청을 전송합니다...\n")

start_time = time.time()
results = []

# ThreadPoolExecutor로 병렬 처리 (최대 3개 동시)
with ThreadPoolExecutor(max_workers=3) as executor:
    futures = {executor.submit(send_request, q, i): i for i, q in enumerate(quick_questions, 1)}
    
    for future in as_completed(futures):
        idx, status, question, success = future.result()
        
        if success and status == 200:
            print(f"✅ [{idx}/{len(quick_questions)}] {question[:30]}... → HTTP {status}")
            results.append(True)
        else:
            print(f"❌ [{idx}/{len(quick_questions)}] {question[:30]}... → Failed")
            results.append(False)

elapsed = time.time() - start_time
success_count = sum(results)
fail_count = len(results) - success_count

print(f"\n{'='*70}")
print(f"📊 부하 테스트 결과:")
print(f"   총 요청: {len(quick_questions)}개")
print(f"   ✅ 성공: {success_count}개")
print(f"   ❌ 실패: {fail_count}개")
print(f"   ⏱️  소요 시간: {elapsed:.1f}초")
print(f"   📈 초당 요청: {len(quick_questions)/elapsed:.1f} req/s")

if success_count > 0:
    print(f"\n🎉 {success_count}개의 Agent 호출 데이터가 생성되었습니다!")
    print(f"\n💡 Application Analytics 확인:")
    print(f"   1. 5-10분 후 Azure AI Foundry 포털 접속")
    print(f"   2. https://ai.azure.com")
    print(f"   3. 프로젝트 → '평가' → 'Application Analytics'")
    print(f"   4. 다음 메트릭이 표시됩니다:")
    print(f"      • Total inference calls: 증가")
    print(f"      • Average inference call duration: 계산됨")
    print(f"      • Error rate: {fail_count}/{len(quick_questions)}")
    print(f"\n   🔄 더 많은 데이터를 원하면 이 셀을 다시 실행하세요!")
else:
    print(f"\n❌ 모든 요청이 실패했습니다.")
    print(f"   Agent Service 상태를 확인하세요.")

print(f"{'='*70}")

=== Application Analytics 데이터 생성 (부하 테스트) ===

📤 8개의 요청을 전송합니다...

✅ [1/8] 1 + 1은?... → HTTP 200
✅ [2/8] 2 * 3은?... → HTTP 200
✅ [3/8] 10 - 5는?... → HTTP 200
✅ [5/8] 현재 시간은?... → HTTP 200
✅ [4/8] 100 / 4는?... → HTTP 200
✅ [6/8] 5 + 5는?... → HTTP 200
✅ [8/8] 20 / 2는?... → HTTP 200
✅ [7/8] 3 * 7은?... → HTTP 200

📊 부하 테스트 결과:
   총 요청: 8개
   ✅ 성공: 8개
   ❌ 실패: 0개
   ⏱️  소요 시간: 46.8초
   📈 초당 요청: 0.2 req/s

🎉 8개의 Agent 호출 데이터가 생성되었습니다!

💡 Application Analytics 확인:
   1. 5-10분 후 Azure AI Foundry 포털 접속
   2. https://ai.azure.com
   3. 프로젝트 → '평가' → 'Application Analytics'
   4. 다음 메트릭이 표시됩니다:
      • Total inference calls: 증가
      • Average inference call duration: 계산됨
      • Error rate: 0/8

   🔄 더 많은 데이터를 원하면 이 셀을 다시 실행하세요!
